In [1]:
import time
import datetime
import pandas as pd
import chromedriver_binary
from tqdm import tqdm
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import numpy as np

pd.set_option("display.max_rows",
              None, 
              "display.max_columns",
              None)

In [2]:
import chromedriver_autoinstaller as chromedriver
chromedriver.install()

'/opt/homebrew/lib/python3.9/site-packages/chromedriver_autoinstaller/111/chromedriver'

In [3]:
browser = webdriver.Chrome()

In [4]:
dates = pd.date_range(start='1/1/2015', end='1/1/2021', freq='15D').format()

start_dates = dates[:-1]
end_dates = dates[1:]

date_intervals = list(zip(start_dates, end_dates))

In [5]:
def html_extractor():

    html=browser.page_source

    soup=BeautifulSoup(html,'html.parser')

    time.sleep(3)

    table = soup.find('table', {'class': 'dataTable no-footer'})

    return table

def scrape(date_intervals_list):
    
    page_cases = []
    failed_search = []

    for i, j in tqdm(date_intervals_list):

        browser.get("https://wcca.wicourts.gov/advanced.html")

        time.sleep(3)

        case_type = browser.find_element(By.XPATH, "//*[@id='react-select-5--value']/div[2]/input")
        case_type.send_keys('Hospital Lien (HL)' + Keys.ENTER)

        start_date = browser.find_element(By.XPATH, '//*[@id="home-container"]/main/div/form/div[6]/div/div/label/div/div/div[1]/div/label/div/div[2]/div/div[2]/div/div/div/input')
        start_date.send_keys(i + Keys.ENTER)

        end_date = browser.find_element(By.XPATH, '//*[@id="home-container"]/main/div/form/div[6]/div/div/label/div/div/div[2]/div/label/div/div[2]/div/div[2]/div/div/div/input')
        end_date.send_keys(j + Keys.ENTER)

        time.sleep(5)

        search = browser.find_element(By.NAME, "search")
        search.click()

        time.sleep(10)

        if i == '2015-01-01':

            entries_btn = browser.find_element(By.XPATH, "//*[@id='caseSearchResults_length']/label/select/option[5]")
            entries_btn.click()

        try:

            extracted_html = html_extractor()

            cases_df = pd.read_html(str(extracted_html), header=0)[0].drop([0])
            
            page_cases.append(cases_df)

        except:

            failed_search.append(i)

            print('Something went wrong trying to scrape: ' + i)
            
            pass

    browser.close()

    all_cases = pd.concat(page_cases)

    print('Failed search: ' + str(failed_search))
    
    return all_cases


def preprocessing(cases_df):

    pp_df = cases_df.drop_duplicates()

    pp_df = pp_df[['Case number', 'County name']]

    pp_df = pp_df.reset_index(drop=True)

    return pp_df

In [6]:
cases_df = scrape(date_intervals[:3])

100%|██████████| 3/3 [01:25<00:00, 28.35s/it]

Failed search: []


In [7]:
cases_df_cleaned = preprocessing(cases_df)

In [8]:
cases_df_cleaned.head()

,Case number,County name
0,2015HL000022,Dane
1,2015HL000021,Dane
2,2015HL000020,Dane
3,2015HL000002,Winnebago
4,2015HL000002,Outagamie


In [11]:
cases_df_cleaned.to_excel('./data/hl_cases_sample.xlsx', index=False)